In [1]:
%matplotlib inline

In [2]:
# Author: Yulou Zhou

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import csv
import glob
import os
import pandas as pd
import numpy as np
import csv
from random import shuffle
torch.manual_seed(1)

In [3]:
path = ''
note = {}
files = glob.glob(os.path.join(path, '*.csv'))
newvar = []

windows = []
for file in files:
    with open(file, 'r') as f:
        #print(file)
        reader = csv.reader(f)
        all_list = list(reader)
        wsize = 64
        for i in range(len(all_list) - wsize):
            windpairs = all_list[i: i+wsize]
            x = [element[0] for element in windpairs]
            y = [element[1] for element in windpairs]
            windows.append((['1', '3', '5'], x, y))

In [4]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    #return idxs
    return torch.tensor(idxs, dtype=torch.long)

word_to_ix={}
shuffle(windows)
training_data = windows[:65536]
for prev, sent, tags in windows:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = int(word)
print(word_to_ix)
tag_to_ix = word_to_ix


# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
PREV_DIM = 3
HIDDEN_DIM = 30

{'1': 1, '12': 12, '8': 8, '6': 6, '3': 3, '10': 10, '0': 0, '5': 5, '7': 7, '2': 2, '11': 11, '4': 4, '9': 9}


Create the model:



In [5]:
class LSTMTagger(nn.Module):

    def __init__(self, batch_size, embedding_dim, hidden_dim, prev_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        #self.chord_embeddings = nn.Embedding(vocab_size, hidden_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (Variable(torch.zeros(1, batch_size, self.hidden_dim)),
                Variable(torch.zeros(1, batch_size, self.hidden_dim)))

    def forward(self, sentence, prev):
        embeds = self.word_embeddings(sentence)
        #chords =  self.chord_embeddings(prev)
        lstm_out, self.hidden = self.lstm(
            embeds.transpose(0,1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.contiguous().view(len(sentence), batch_size, -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Train the model:



In [6]:
from torch.autograd import Variable

batch_size = 256
model = LSTMTagger(batch_size, EMBEDDING_DIM, HIDDEN_DIM, PREV_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.2)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
# with torch.no_grad():
#     inputs = prepare_sequence(training_data[0][1], word_to_ix)
#     prevs = prepare_sequence(training_data[0][0], word_to_ix)
#     tag_scores = model(inputs, prevs)
#     _, point = tag_scores.max(1)
#     print(point)


for epoch in range(500):  # again, normally you would NOT do 300 epochs, it is toy data
    shuffle(training_data)
    sentence_in = (torch.stack([prepare_sequence(a[1], word_to_ix) for a in training_data[:batch_size]])).transpose(0,1)
    #print(sentence_in.size())
    prevs_in = (torch.stack([prepare_sequence(a[0], word_to_ix) for a in training_data[:batch_size]])).transpose(0,1)
    targets = (torch.stack([prepare_sequence(a[2], tag_to_ix) for a in training_data[:batch_size]])).transpose(0,1)
    prevs_in, sentence_in, targets = Variable(prevs_in), Variable(sentence_in), Variable(targets)

    model.zero_grad()
        
    model.hidden = model.init_hidden()
    
    tag_scores = model(sentence_in, prevs_in)
    if epoch % 30 == 0:
        #print(tag_scores.size())
        _, point = tag_scores.max(2)
        #print("point:", point[0])
        #print("pointt:", (point.transpose(0,1))[0])

    # Step 4. Compute the loss, gradients, and update the parameters by
    #  calling optimizer.step()
    #print((tag_scores.contiguous().view(-1, 13)).size(), (targets.contiguous().view(-1)).size())
    loss = loss_function(tag_scores.contiguous().view(-1, 13), targets.contiguous().view(-1))
    loss.backward()
    optimizer.step()
    
    #TODO: modify according to https://cs230-stanford.github.io/pytorch-getting-started.html#training-vs-evaluation

#     for prev, sentence, tags in training_data[:32]:
        
#         # Step 1. Remember that Pytorch accumulates gradients.
#         # We need to clear them out before each instance
#         model.zero_grad()
        
#         # Also, we need to clear out the hidden state of the LSTM,
#         # detaching it from its history on the last instance.
#         model.hidden = model.init_hidden()

#         # Step 2. Get our inputs ready for the network, that is, turn them into
#         # Tensors of word indices.
#         sentence_in = prepare_sequence(sentence, word_to_ix)
#         prevs_in = prepare_sequence(prev, word_to_ix)
#             #print("prevs_in ", prevs_in)
#         targets = prepare_sequence(tags, tag_to_ix)
        
#         # Step 3. Run our forward pass.
#         tag_scores = model(sentence_in, prevs_in)

#         # Step 4. Compute the loss, gradients, and update the parameters by
#         #  calling optimizer.step()
#         loss = loss_function(tag_scores, targets)
#         loss.backward()
#         optimizer.step()
    if(epoch%30==0): print(loss)


# See what the scores are after training
# with torch.no_grad():
#     inputs = prepare_sequence(training_data[0][0], word_to_ix)
#     tag_scores = model(inputs)

#     # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
#     # for word i. The predicted tag is the maximum scoring tag.
#     # Here, we can see the predicted sequence below is 0 1 2 0 1
#     # since 0 is index of the maximum value of row 1,
#     # 1 is the index of maximum value of row 2, etc.
#     # Which is DET NOUN VERB DET NOUN, the correct sequence!
#     _, point2 = tag_scores.max(1)
#     print(point2)

tensor(5.5462)
tensor(5.5472)
tensor(5.5467)
tensor(5.5462)


KeyboardInterrupt: 

In [ ]:
accuracy = 0.0
c = 0
for i in range(100):
    shuffle(training_data)
    with torch.no_grad():
        inputs = prepare_sequence(training_data[0][1], word_to_ix)
        prevsa = prepare_sequence(training_data[0][0], word_to_ix)
        tag_scores = model(inputs, prevsa)

        # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
        # for word i. The predicted tag is the maximum scoring tag.
        # Here, we can see the predicted sequence below is 0 1 2 0 1
        # since 0 is index of the maximum value of row 1,
        # 1 is the index of maximum value of row 2, etc.
        # Which is DET NOUN VERB DET NOUN, the correct sequence!
        _, point2 = tag_scores.max(1)
        #print(point2.tolist())

    values, indices = tag_scores.max(1)
    #print(training_data[0][0])
    #print(training_data[0][1][10:20])
    #print(training_data[0][2][10:20])
    truth = np.array([int(a) for a in training_data[0][2]])
    predicted = point2.numpy()
    print(truth[:10], predicted[:10])
    accuracy += np.average((truth == predicted))
    c += 1
print(accuracy/c)